# Assembly

Assembly is defined by an attempt to reverse engineer a genome. We are given overlapping pieces as the result sequencing, and we must rely on algorithms to combine these overlapping pieces together. In this demonstration, we will attempt to demonstrate why this problem is both a computational and biological challenge.

First we will start with a function that returns the k-mers for a text string.

In [1]:
def composition(k,text):
    patterns = []
    # YOUR SOLUTION HERE
    ## BEGIN SOLUTION
    for i in range(0,len(text)-k+1):
        patterns.append(text[i:(i+k)])
    patterns = sorted(patterns)
    ## END SOLUTION
    return patterns

In [2]:
composition(3,"TATGGGGTGC")

['ATG', 'GGG', 'GGG', 'GGT', 'GTG', 'TAT', 'TGC', 'TGG']

Modern assemblers rely on a de Bruijn graph constructed from the $k$-mer composition. We will briefly review this graph.

## Constructing de Bruijn graphs from $k$-mer composition
Given a collection of $k$-mers Patterns, the nodes of de_bruijn(k,patterns) graphs are simply all unique (k−1)-mers occurring as a prefix or suffix in Patterns. For example, say we are given the following collection of 3-mers:
<pre>
AAT   ATG   ATG   ATG    CAT   CCA   GAT   GCC   GGA   GGG   GTT   TAA   TGC   TGG   TGT
</pre>
Then the set of eleven unique 2-mers is:
<pre>
AA   AT   CA   CC   GA   GC   GG   GT   TA   TG   TT
</pre>

For every $k$-mer in ``patterns`` we will connect its prefix node to its suffix node by a directed edge in order to produce our final graph.

<img src="http://bioinformaticsalgorithms.com/images/Assembly/debruijn_graph_alternate_rendering.png">

**Stop and think:** Can you make the de Bruijn graph from the k-mers we first created?

In [3]:
kmers = composition(3,"TATGGGGTGC")
kmers

['ATG', 'GGG', 'GGG', 'GGT', 'GTG', 'TAT', 'TGC', 'TGG']

Here is a function the constructs the graph for us.

In [4]:
import assembly_helper

dB = assembly_helper.de_bruijn(kmers)

Uncomment the following line by removing the ``#`` and run the cell below to see if your graph matches

In [5]:
#assembly_helper.show(dB)

We will now fast forward to use a simplistic implementation of a modern assembler. It solves our inverse problem of converting our kmers to an assembly.

In [6]:
seq="TATGGGGTGC"
kmers = composition(3,seq)
assembled_seq=assembly_helper.reconstruct(kmers)
assembled_seq

'TATGGGGTGC'

We have an alignment algorithm so we can check it out:

In [7]:
sc,s1,s2=assembly_helper.align_dynamic2(seq,assembled_seq)
assembly_helper.print_alignment(s1,s2)

TATGGGGTGC
||||||||||
TATGGGGTGC


Looks like we solved it! Time to go home...

Let's take a look at Yeast's genome. We will read in all the chromosomes, but for simplicity we will focus on the first 1,000 nucleotides of the first chromosome.

In [8]:
headers,seqs=assembly_helper.read_fasta('GCF_000146045.2_R64_genomic.fna')
headers[0]

'>NC_001133.9 Saccharomyces cerevisiae S288C chromosome I, complete sequence'

In [11]:
ch1_first_1000 = seqs[0][:1000].upper()

In [12]:
ch1_first_1000

'CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACCCACACACACACATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTGGCCAACCTGTCTCTCAACTTACCCTCCATTACCCTGCCTCCACTCGTTACCCTGTCCCATTCAACCATACCACTCCGAACCACCATCCATCCCTCTACTTACTACCACTCACCCACCGTTACCCTCCAATTACCCATATCCAACCCACTGCCACTTACCCTACCATTACCCTACCATCCACCATGACCTACTCACCATACTGTTCTTCTACCCACCATATTGAAACGCTAACAAATGATCGTAAATAACACACACGTGCTTACCCTACCACTTTATACCACCACCACATGCCATACTCACCCTCACTTGTATACTGATTTTACGTACGCACACGGATGCTACAGTATATACCATCTCAAACTTACCCTACTCTCAGATTCCACTTCACTCCATGGCCCATCTCTCACTGAATCAGTACCAAATGCACTCACATCATTATGCACGGCACTTGCCTCAGCGGTCTATACCCTGTGCCATTTACCCATAACGCCCATCATTATCCACATTTTGATATCTATATCTCATTCGGCGGTCCCAAATATTGTATAACTGCCCTTAATACATACGTTATACCACTTTTGCACCATATACTTACCACTCCATTTATATACACTTATGTCAATATTACAGAAAAATCCCCACAAAAATCACCTAAACATAAAAATATTCTACTTTTCAACAATAATACATAAACATATTGGCTTGTGGTAGCAACACTATCATGGTATCACTAACGTAAAAGTTCCTCAATATTGCAATTTGCTTGAACGGATGCTATTTCAGAATATTTCGTACTTACACAGGCCATACATTAGAATAATATGTCACATCACTGTCGTAACACTCTTTATTCACCGAGCAATAATACGGTAGTGGCTCAAACTCATGCGGGTGCTATG

**Stop and think:** Do you think we will get the correct assembly using the same parameters and functions that were so successful above?

In [18]:
kmers = composition(3,ch1_first_1000)
assembled_seq=assembly_helper.reconstruct(kmers)
assembled_seq

'CCTGATGATGATGATGATGATCGTAGTAGTAGTACTGAGTACTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCGTACTCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATACTCATACTCATACTCATACTCATACTCATACTCATACTCATACTCATACTACTACTACTACTACTACTACTACTACTACTACTACTACTACTACTACTACTACTACTACTACTACTACGGGCCGGCCGGCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCATACGGCCATACGGCCATACGGCCATACGGCATACGGACGCATACGCATACGCATACGAATACGAATAATAATAATAATAATAATAATAATAATAATAATAATAATAATAATAATAATAATAAGGAACCAGCAGCAGCAGCAGAACCAGAACCAGAACCAGAACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACCACACACACACACACACACACACACACACACACACACACACACACACACACAACAACAACAACAACAACAACAACAACAACAACAACAACAACAACAACAACAAAAAAAAAAAAAAAAAAAAAATTTTTTTTTTTTTTTTTGTTGTTGTTGTTGTTGTGTGTGTGTGGTCTTGGTCTTGGTCTTGGTCTTGGTCTTGGTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCGTCGTATTCGTATTCGTATTATTATTATTATTATTATTATTATTATTATTATTATTATTATTATTATTATTATTATTATGCTTATGCTTATGCTTATGCTTATGCTGCTGCTGCTGCGTATGCGTATGCGTATGCCTGCCTGCCTGCCTGCCTG

In [19]:
assembly_helper.print_alignment(ch1_first_1000,assembled_seq,num_to_print=50)

CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACC
||  |  |     |                |  ||  |  || || |  |
CCTGATGATGATGATGATGATCGTAGTAGTAGTACTGAGTACTCCTCCTC
CACACACACACATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTG
| |   | |     |   | |  ||    |     |||  ||    |   
CTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCGTACTCA
GCCAACCTGTCTCTCAACTTACCCTCCATTACCCTGCCTCCACTCGTTAC
 |       ||       | | | ||     |     |  ||       |
TCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATC
CCTGTCCCATTCAACCATACCACTCCGAACCACCATCCATCCCTCTACTT
    ||  | |||  |  |          |   |    |  | |   | |
ATCATCATACTCATACTCATACTCATACTCATACTCATACTCATACTCAT
ACTACCACTCACCCACCGTTACCCTCCAATTACCCATATCCAACCCACTG
|||   |||    |  |     |  | | |    |   | | ||      
ACTCATACTACTACTACTACTACTACTACTACTACTACTACTACTACTAC
CCACTTACCCTACCATTACCCTACCATCCACCATGACCTACTCACCATAC
        |      |    |||| |    ||        | | ||   |
TACTACTACTACTACTACTACTACTACGGGCCGGCCGGCCCCCCCCCCCC
TGTTCTTCTACCCACCATATTGAAACGCTAACAAATGATCGTAAATAACA
    |  |  ||| ||    |    |  || 

How does that look to you? Take a look at all C's in a row. And all the T's and the altnerating AC's. Ahhh....

The real question is not whether this assembly is wrong (it clearly is), but the question is what caused this problem? There are a number of things to consider:
1. It could be a bug in our program (it isn't :)
2. It could be that our program returned one of many possible assemblies (ding ding ding!). 

It is the second complication. With a small $k$-mer value we run the risk of too much variability in the assemblies that are possible. 

**Stop and experiment:** See how the results change as you increase the $k$ value:

In [20]:
kmers=composition(11,ch1_first_1000)
assembled_seq=assembly_helper.reconstruct(kmers)
assembled_seq

'CCACACCACACCCACACACCCACACACCACACCACACCACACACCACACCCACACACACACATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTGGCCAACCTGTCTCTCAACTTACCCTCCATTACCCTGCCTCCACTCGTTACCCTGTCCCATTCAACCATACCACTCCGAACCACCATCCATCCCTCTACTTACTACCACTCACCCACCGTTACCCTCCAATTACCCATATCCAACCCACTGCCACTTACCCTACCATTACCCTACCATCCACCATGACCTACTCACCATACTGTTCTTCTACCCACCATATTGAAACGCTAACAAATGATCGTAAATAACACACACGTGCTTACCCTACCACTTTATACCACCACCACATGCCATACTCACCCTCACTTGTATACTGATTTTACGTACGCACACGGATGCTACAGTATATACCATCTCAAACTTACCCTACTCTCAGATTCCACTTCACTCCATGGCCCATCTCTCACTGAATCAGTACCAAATGCACTCACATCATTATGCACGGCACTTGCCTCAGCGGTCTATACCCTGTGCCATTTACCCATAACGCCCATCATTATCCACATTTTGATATCTATATCTCATTCGGCGGTCCCAAATATTGTATAACTGCCCTTAATACATACGTTATACCACTTTTGCACCATATACTTACCACTCCATTTATATACACTTATGTCAATATTACAGAAAAATCCCCACAAAAATCACCTAAACATAAAAATATTCTACTTTTCAACAATAATACATAAACATATTGGCTTGTGGTAGCAACACTATCATGGTATCACTAACGTAAAAGTTCCTCAATATTGCAATTTGCTTGAACGGATGCTATTTCAGAATATTTCGTACTTACACAGGCCATACATTAGAATAATATGTCACATCACTGTCGTAACACTCTTTATTCACCGAGCAATAATACGGTAGTGGCTCAAACTCATGCGGGTGCTATG

In [21]:
assembly_helper.print_alignment(ch1_first_1000,assembled_seq,num_to_print=50)

CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACC
|||||||||||||||||||||||||||||||||||||  |||||||||||
CCACACCACACCCACACACCCACACACCACACCACACCACACACCACACC
CACACACACACATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTG
||||||||||||||||||||||||||||||||||||||||||||||||||
CACACACACACATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTG
GCCAACCTGTCTCTCAACTTACCCTCCATTACCCTGCCTCCACTCGTTAC
||||||||||||||||||||||||||||||||||||||||||||||||||
GCCAACCTGTCTCTCAACTTACCCTCCATTACCCTGCCTCCACTCGTTAC
CCTGTCCCATTCAACCATACCACTCCGAACCACCATCCATCCCTCTACTT
||||||||||||||||||||||||||||||||||||||||||||||||||
CCTGTCCCATTCAACCATACCACTCCGAACCACCATCCATCCCTCTACTT
ACTACCACTCACCCACCGTTACCCTCCAATTACCCATATCCAACCCACTG
||||||||||||||||||||||||||||||||||||||||||||||||||
ACTACCACTCACCCACCGTTACCCTCCAATTACCCATATCCAACCCACTG
CCACTTACCCTACCATTACCCTACCATCCACCATGACCTACTCACCATAC
||||||||||||||||||||||||||||||||||||||||||||||||||
CCACTTACCCTACCATTACCCTACCATCCACCATGACCTACTCACCATAC
TGTTCTTCTACCCACCATATTGAAACGCTAACAAATGATCGTAAATAACA
|||||||||||||||||||||||||||||||

Well that's better! But there are still a couple of errors. Try increasing the k-value and see if we can correct the problem.

In [22]:
kmers=composition(13,ch1_first_1000)
assembled_seq=assembly_helper.reconstruct(kmers)
assembled_seq

'CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACCCACACACACACATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTGGCCAACCTGTCTCTCAACTTACCCTCCATTACCCTGCCTCCACTCGTTACCCTGTCCCATTCAACCATACCACTCCGAACCACCATCCATCCCTCTACTTACTACCACTCACCCACCGTTACCCTCCAATTACCCATATCCAACCCACTGCCACTTACCCTACCATTACCCTACCATCCACCATGACCTACTCACCATACTGTTCTTCTACCCACCATATTGAAACGCTAACAAATGATCGTAAATAACACACACGTGCTTACCCTACCACTTTATACCACCACCACATGCCATACTCACCCTCACTTGTATACTGATTTTACGTACGCACACGGATGCTACAGTATATACCATCTCAAACTTACCCTACTCTCAGATTCCACTTCACTCCATGGCCCATCTCTCACTGAATCAGTACCAAATGCACTCACATCATTATGCACGGCACTTGCCTCAGCGGTCTATACCCTGTGCCATTTACCCATAACGCCCATCATTATCCACATTTTGATATCTATATCTCATTCGGCGGTCCCAAATATTGTATAACTGCCCTTAATACATACGTTATACCACTTTTGCACCATATACTTACCACTCCATTTATATACACTTATGTCAATATTACAGAAAAATCCCCACAAAAATCACCTAAACATAAAAATATTCTACTTTTCAACAATAATACATAAACATATTGGCTTGTGGTAGCAACACTATCATGGTATCACTAACGTAAAAGTTCCTCAATATTGCAATTTGCTTGAACGGATGCTATTTCAGAATATTTCGTACTTACACAGGCCATACATTAGAATAATATGTCACATCACTGTCGTAACACTCTTTATTCACCGAGCAATAATACGGTAGTGGCTCAAACTCATGCGGGTGCTATG

In [23]:
assembly_helper.print_alignment(ch1_first_1000,assembled_seq,num_to_print=50)

CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACC
||||||||||||||||||||||||||||||||||||||||||||||||||
CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACC
CACACACACACATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTG
||||||||||||||||||||||||||||||||||||||||||||||||||
CACACACACACATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTG
GCCAACCTGTCTCTCAACTTACCCTCCATTACCCTGCCTCCACTCGTTAC
||||||||||||||||||||||||||||||||||||||||||||||||||
GCCAACCTGTCTCTCAACTTACCCTCCATTACCCTGCCTCCACTCGTTAC
CCTGTCCCATTCAACCATACCACTCCGAACCACCATCCATCCCTCTACTT
||||||||||||||||||||||||||||||||||||||||||||||||||
CCTGTCCCATTCAACCATACCACTCCGAACCACCATCCATCCCTCTACTT
ACTACCACTCACCCACCGTTACCCTCCAATTACCCATATCCAACCCACTG
||||||||||||||||||||||||||||||||||||||||||||||||||
ACTACCACTCACCCACCGTTACCCTCCAATTACCCATATCCAACCCACTG
CCACTTACCCTACCATTACCCTACCATCCACCATGACCTACTCACCATAC
||||||||||||||||||||||||||||||||||||||||||||||||||
CCACTTACCCTACCATTACCCTACCATCCACCATGACCTACTCACCATAC
TGTTCTTCTACCCACCATATTGAAACGCTAACAAATGATCGTAAATAACA
|||||||||||||||||||||||||||||||

So now our question becomes, is this it? Can we call it a day? Well don't forget that Yeast has more than one chromosome. If you construct kmers from two chromosomes, then our algorithm falls apart and needs significant improvements.

In [25]:
ch2_first_30 = seqs[1][:30].upper()
ch2_first_30

'AAATAGCCCTCATGTACGTCTCCTCCAAGC'

**Warning:** This is supposed to produce an error as the graph produced by combining the kmers from the two chromosomes is not structured correctly for our assembler.

In [26]:
kmers=composition(13,ch1_first_1000)+composition(13,ch2_first_30)
assembled_seq=assembly_helper.reconstruct(kmers)
assembled_seq

KeyError: 'TCTCCTCCAAGC'

Now that you've gone through one run of the notebook, feel free to play around with any of the parameters!